<h1><center>MPST: A Corpus of Movie Plot Synopses with Tags</center></h1>

In [1]:
import pandas as pd
import sqlite3
import os
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns


from wordcloud import WordCloud
import numpy as np
import re

from bs4 import BeautifulSoup
import tqdm

import nltk
from itertools import combinations
#from toolz import compose

from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')

from nltk.corpus import words
from nltk.tokenize import sent_tokenize
stemmer = SnowballStemmer('english')
from scipy.sparse import coo_matrix, hstack
from tqdm import tqdm

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import f1_score,precision_score,recall_score,hamming_loss

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MultiLabelBinarizer

import xgboost as xgb

[nltk_data] Downloading package punkt to
[nltk_data]     /home/krushithreddy0817/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data = pd.read_csv("cleaned_data.csv")
data.head()

,title,plot_synopsis,tags,split,synopsis_source,cnt_dup,tag_count,synopsis_count,synopsis_sent_count,CleanedSynopsis
0,$,"Set in Hamburg, West Germany, several criminal...",murder,test,imdb,1,1,648,26,set hamburg west germani sever crimin take adv...
1,$windle,A 6th grader named Griffin Bing decides to gat...,flashback,train,wikipedia,1,1,353,14,grader name griffin bing decid gather entir gr...
2,'71,"Gary Hook, a new recruit to the British Army, ...","suspenseful, neo noir, murder, violence",train,wikipedia,1,4,699,39,gari hook new recruit british armi take leav m...
3,'A' gai wak,Sergeant Dragon Ma (Jackie Chan) is part of th...,"cult, violence",train,wikipedia,1,2,665,41,sergeant dragon jacki chan part hong kong mari...
4,'Breaker' Morant,"In Pretoria, South Africa, in 1902, Major Char...","murder, anti war, violence, flashback, tragedy...",train,wikipedia,1,6,1694,140,pretoria south africa major charl bolton rod m...


In [3]:
conn = sqlite3.connect('cleaned_data.db')
data.to_sql('cleaned_data', conn, if_exists='replace', index=False)
train = pd.read_sql("Select * From cleaned_data where split = 'train' OR split='val'",conn)
test =  pd.read_sql("Select * From cleaned_data where split = 'test'",conn)
conn.close()

In [5]:
print(train.head())
print(train.shape)

              title                                      plot_synopsis  \
0           $windle  A 6th grader named Griffin Bing decides to gat...   
1               '71  Gary Hook, a new recruit to the British Army, ...   
2       'A' gai wak  Sergeant Dragon Ma (Jackie Chan) is part of th...   
3  'Breaker' Morant  In Pretoria, South Africa, in 1902, Major Char...   
4           'C'-Man  Customs Investigator Cliff Holden (Dean Jagger...   

                                                tags  split synopsis_source  \
0                                          flashback  train       wikipedia   
1            suspenseful, neo noir, murder, violence  train       wikipedia   
2                                     cult, violence  train       wikipedia   
3  murder, anti war, violence, flashback, tragedy...  train       wikipedia   
4                                             murder  train            imdb   

   cnt_dup  tag_count  synopsis_count  synopsis_sent_count  \
0        1        

In [11]:
print(test.head())
print(test.shape)

                                             title  \
0                                                $   
1                            'Crocodile' Dundee II   
2                      'Hukkunud Alpinisti' hotell   
3  'Northwest Passage' (Book I -- Rogers' Rangers)   
4                             (500) Days of Summer   

                                       plot_synopsis  \
0  Set in Hamburg, West Germany, several criminal...   
1  A year has passed since the events of Crocodil...   
2  Inspector Glebsky arrives at the hotel "Dead M...   
3  === Book 1 ===\nLangdon Towne is a young Congr...   
4  (500) Days of Summer is presented in a non-chr...   

                                                tags split synopsis_source  \
0                                             murder  test            imdb   
1                                             murder  test       wikipedia   
2                             cult, neo noir, murder  test       wikipedia   
3           tragedy, reven

# Tag vectorizaton:

In [4]:
tag_vect = CountVectorizer(max_features=3,tokenizer = lambda x: str(x).split(", "))
y_train = tag_vect.fit_transform(train['tags'])
y_test = tag_vect.transform(test['tags'])
print(tag_vect.get_feature_names())

['flashback', 'murder', 'violence']


In [5]:
names = tag_vect.get_feature_names()

# CountVectorizing with words and uni grams:

In [6]:
vectorize=CountVectorizer()
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

In [7]:
print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 78666)   (11027, 3)
(2730, 78666)   (2730, 3)


In [8]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  35 out of  45 | elapsed:    3.8s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  40 out of  45 | elapsed:    4.2s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    4.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    4.7s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.5320886096806409


In [9]:
from sklearn.metrics import classification_report
clf = OneVsRestClassifier(MultinomialNB(alpha=1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5143, Recall: 0.5577, F1-measure: 0.5351
              precision    recall  f1-score   support

   flashback       0.27      0.25      0.26       549
      murder       0.60      0.66      0.63      1043
    violence       0.55      0.63      0.58       827

   micro avg       0.51      0.56      0.54      2419
   macro avg       0.47      0.51      0.49      2419
weighted avg       0.51      0.56      0.53      2419
 samples avg       0.31      0.33      0.30      2419



In [10]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    4.9s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    5.5s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    5.5s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]


In [11]:
grid_search_cv.best_score_

0.5299295701288367

In [12]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6351, Recall: 0.4188, F1-measure: 0.5047
              precision    recall  f1-score   support

   flashback       0.39      0.12      0.18       549
      murder       0.68      0.55      0.61      1043
    violence       0.64      0.45      0.53       827

   micro avg       0.64      0.42      0.50      2419
   macro avg       0.57      0.37      0.44      2419
weighted avg       0.60      0.42      0.48      2419
 samples avg       0.27      0.24      0.24      2419



In [13]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    3.4s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    3.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    3.9s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.5297687098911606


In [14]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=0.1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6327, Recall: 0.4237, F1-measure: 0.5076
              precision    recall  f1-score   support

   flashback       0.37      0.11      0.17       549
      murder       0.70      0.53      0.61      1043
    violence       0.62      0.49      0.55       827

   micro avg       0.63      0.42      0.51      2419
   macro avg       0.56      0.38      0.44      2419
weighted avg       0.60      0.42      0.49      2419
 samples avg       0.27      0.24      0.24      2419



# CountVectorizing with words and bi grams:

In [15]:
vectorize=CountVectorizer(ngram_range=(2,2),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 89722)   (11027, 3)
(2730, 89722)   (2730, 3)


In [18]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,2.5,2,1.5,1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  35 out of  45 | elapsed:    2.2s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  40 out of  45 | elapsed:    2.4s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    2.4s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=2, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.5573266063061614


In [19]:
clf = OneVsRestClassifier(MultinomialNB(alpha=2),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5723, Recall: 0.5250, F1-measure: 0.5476
              precision    recall  f1-score   support

   flashback       0.36      0.16      0.22       549
      murder       0.64      0.67      0.65      1043
    violence       0.55      0.59      0.57       827

   micro avg       0.57      0.53      0.55      2419
   macro avg       0.52      0.47      0.48      2419
weighted avg       0.54      0.53      0.53      2419
 samples avg       0.30      0.30      0.29      2419



In [21]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6,10**-7,10**-8],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    1.5s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    1.6s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    1.8s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.48161720542090897


In [22]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5409, Recall: 0.3907, F1-measure: 0.4537
              precision    recall  f1-score   support

   flashback       0.31      0.17      0.22       549
      murder       0.62      0.49      0.54      1043
    violence       0.55      0.42      0.47       827

   micro avg       0.54      0.39      0.45      2419
   macro avg       0.49      0.36      0.41      2419
weighted avg       0.52      0.39      0.45      2419
 samples avg       0.25      0.22      0.22      2419



In [23]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    2.2s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    2.3s remaining:    0.1s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.48235447208338217


[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    2.4s finished


In [24]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5190, Recall: 0.4456, F1-measure: 0.4795
              precision    recall  f1-score   support

   flashback       0.29      0.18      0.22       549
      murder       0.59      0.55      0.57      1043
    violence       0.53      0.49      0.51       827

   micro avg       0.52      0.45      0.48      2419
   macro avg       0.47      0.41      0.43      2419
weighted avg       0.50      0.45      0.47      2419
 samples avg       0.27      0.25      0.25      2419



# CountVectorizing with words and tri grams:

In [25]:
vectorize=CountVectorizer(ngram_range=(3,3),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 5037)   (11027, 3)
(2730, 5037)   (2730, 3)


In [26]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1396s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  15 out of  40 | elapsed:    0.3s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  20 out of  40 | elapsed:    0.5s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    0.6s remaining:    0.2s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.4046109161812072


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.8s finished


In [27]:
clf = OneVsRestClassifier(MultinomialNB(alpha=1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4980, Recall: 0.3129, F1-measure: 0.3844
              precision    recall  f1-score   support

   flashback       0.31      0.17      0.22       549
      murder       0.53      0.49      0.51      1043
    violence       0.57      0.19      0.29       827

   micro avg       0.50      0.31      0.38      2419
   macro avg       0.47      0.28      0.34      2419
weighted avg       0.50      0.31      0.37      2419
 samples avg       0.20      0.18      0.18      2419



In [28]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0693s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  40 | elapsed:    0.1s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  15 out of  40 | elapsed:    0.2s remaining:    0.3s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.3809500587263854


[Parallel(n_jobs=-1)]: Done  20 out of  40 | elapsed:    0.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.4s finished


In [29]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4137, Recall: 0.3439, F1-measure: 0.3756
              precision    recall  f1-score   support

   flashback       0.26      0.21      0.23       549
      murder       0.48      0.40      0.44      1043
    violence       0.42      0.36      0.39       827

   micro avg       0.41      0.34      0.38      2419
   macro avg       0.39      0.32      0.35      2419
weighted avg       0.41      0.34      0.38      2419
 samples avg       0.21      0.20      0.19      2419



In [30]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0490s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   5 out of  40 | elapsed:    0.1s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  10 out of  40 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  15 out of  40 | elapsed:    0.1s remaining:    0.2s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.3839222235151611


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.4s finished


In [31]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.3991, Recall: 0.3712, F1-measure: 0.3847
              precision    recall  f1-score   support

   flashback       0.26      0.21      0.24       549
      murder       0.47      0.45      0.46      1043
    violence       0.39      0.38      0.38       827

   micro avg       0.40      0.37      0.38      2419
   macro avg       0.37      0.35      0.36      2419
weighted avg       0.40      0.37      0.38      2419
 samples avg       0.22      0.21      0.20      2419



# CountVectorizing with words and uni,bi grams:

In [32]:
vectorize=CountVectorizer(ngram_range=(1,2),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 109114)   (11027, 3)
(2730, 109114)   (2730, 3)


In [33]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    4.4s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    4.7s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    5.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    5.2s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.5582516700684903


In [34]:
clf = OneVsRestClassifier(MultinomialNB(alpha=1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5495, Recall: 0.5717, F1-measure: 0.5604
              precision    recall  f1-score   support

   flashback       0.31      0.26      0.28       549
      murder       0.65      0.68      0.66      1043
    violence       0.56      0.64      0.60       827

   micro avg       0.55      0.57      0.56      2419
   macro avg       0.50      0.53      0.51      2419
weighted avg       0.54      0.57      0.55      2419
 samples avg       0.31      0.33      0.30      2419



In [35]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    4.7s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    5.6s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    5.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    5.9s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.5242579145967816


In [36]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6507, Recall: 0.4212, F1-measure: 0.5114
              precision    recall  f1-score   support

   flashback       0.41      0.08      0.13       549
      murder       0.67      0.59      0.63      1043
    violence       0.66      0.43      0.52       827

   micro avg       0.65      0.42      0.51      2419
   macro avg       0.58      0.37      0.43      2419
weighted avg       0.61      0.42      0.48      2419
 samples avg       0.27      0.24      0.25      2419



In [37]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    3.4s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    4.1s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    4.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    4.3s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.5292699952814558


In [39]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=0.1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6192, Recall: 0.4274, F1-measure: 0.5057
              precision    recall  f1-score   support

   flashback       0.35      0.16      0.22       549
      murder       0.67      0.62      0.64      1043
    violence       0.67      0.37      0.47       827

   micro avg       0.62      0.43      0.51      2419
   macro avg       0.56      0.38      0.45      2419
weighted avg       0.60      0.43      0.49      2419
 samples avg       0.28      0.25      0.25      2419



# CountVectorizing with words and uni,bi,tri grams:

In [40]:
vectorize=CountVectorizer(ngram_range=(1,3),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 114151)   (11027, 3)
(2730, 114151)   (2730, 3)


In [41]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    4.7s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    5.2s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    5.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    5.6s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.5577807704762309


In [42]:
clf = OneVsRestClassifier(MultinomialNB(alpha=1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5517, Recall: 0.5692, F1-measure: 0.5603
              precision    recall  f1-score   support

   flashback       0.32      0.27      0.29       549
      murder       0.64      0.67      0.66      1043
    violence       0.56      0.63      0.60       827

   micro avg       0.55      0.57      0.56      2419
   macro avg       0.51      0.53      0.52      2419
weighted avg       0.54      0.57      0.55      2419
 samples avg       0.31      0.33      0.30      2419



In [43]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    5.4s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    6.4s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    6.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    6.6s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.01, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.5195591656876584


In [44]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.01,n_jobs=-1))
clf.fit(X_train,y_train)

predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5530, Recall: 0.4510, F1-measure: 0.4968
              precision    recall  f1-score   support

   flashback       0.30      0.16      0.21       549
      murder       0.62      0.56      0.59      1043
    violence       0.56      0.51      0.53       827

   micro avg       0.55      0.45      0.50      2419
   macro avg       0.50      0.41      0.44      2419
weighted avg       0.53      0.45      0.48      2419
 samples avg       0.28      0.26      0.25      2419



In [45]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    3.3s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    4.0s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    4.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    4.2s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.519281737313241


In [46]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=0.1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6505, Recall: 0.3886, F1-measure: 0.4865
              precision    recall  f1-score   support

   flashback       0.35      0.07      0.12       549
      murder       0.70      0.52      0.60      1043
    violence       0.64      0.43      0.52       827

   micro avg       0.65      0.39      0.49      2419
   macro avg       0.57      0.34      0.41      2419
weighted avg       0.60      0.39      0.46      2419
 samples avg       0.26      0.22      0.23      2419



# CountVectorizing with chars and tri grams:

In [47]:
vectorize=CountVectorizer(ngram_range=(3,3),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 10726)   (11027, 3)
(2730, 10726)   (2730, 3)


In [73]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:    9.8s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   10.9s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   11.2s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=2.5, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.4907140280954945


In [48]:
clf = OneVsRestClassifier(MultinomialNB(alpha=2.5),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4919, Recall: 0.5891, F1-measure: 0.5361
              precision    recall  f1-score   support

   flashback       0.26      0.35      0.30       549
      murder       0.61      0.65      0.63      1043
    violence       0.53      0.67      0.59       827

   micro avg       0.49      0.59      0.54      2419
   macro avg       0.47      0.56      0.51      2419
weighted avg       0.50      0.59      0.54      2419
 samples avg       0.30      0.34      0.30      2419



In [49]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    9.9s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   11.7s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   12.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   12.9s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=2.5, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4996760208324402


In [50]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=2.5,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6706, Recall: 0.3258, F1-measure: 0.4385
              precision    recall  f1-score   support

   flashback       0.20      0.01      0.01       549
      murder       0.70      0.47      0.56      1043
    violence       0.65      0.36      0.46       827

   micro avg       0.67      0.33      0.44      2419
   macro avg       0.51      0.28      0.34      2419
weighted avg       0.57      0.33      0.40      2419
 samples avg       0.21      0.19      0.19      2419



In [53]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   10.2s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   10.5s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   11.5s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.5025470908756237


In [54]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.0001,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4860, Recall: 0.5006, F1-measure: 0.4932
              precision    recall  f1-score   support

   flashback       0.27      0.23      0.25       549
      murder       0.63      0.49      0.55      1043
    violence       0.48      0.70      0.57       827

   micro avg       0.49      0.50      0.49      2419
   macro avg       0.46      0.47      0.45      2419
weighted avg       0.49      0.50      0.49      2419
 samples avg       0.30      0.29      0.28      2419



# CountVectorizing with chars and four grams:

In [55]:
vectorize=CountVectorizer(ngram_range=(4,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 82528)   (11027, 3)
(2730, 82528)   (2730, 3)


In [56]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   16.2s remaining:    3.6s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   18.5s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   19.2s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=2, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.5457335895170159


In [57]:
clf = OneVsRestClassifier(MultinomialNB(alpha=2),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5200, Recall: 0.5763, F1-measure: 0.5467
              precision    recall  f1-score   support

   flashback       0.29      0.30      0.30       549
      murder       0.62      0.66      0.64      1043
    violence       0.55      0.66      0.60       827

   micro avg       0.52      0.58      0.55      2419
   macro avg       0.48      0.54      0.51      2419
weighted avg       0.52      0.58      0.54      2419
 samples avg       0.30      0.33      0.30      2419



In [58]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   19.2s remaining:    6.4s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   23.4s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   23.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   23.5s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.5228850110818296


In [59]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5449, Recall: 0.5345, F1-measure: 0.5396
              precision    recall  f1-score   support

   flashback       0.34      0.17      0.23       549
      murder       0.60      0.65      0.62      1043
    violence       0.54      0.63      0.58       827

   micro avg       0.54      0.53      0.54      2419
   macro avg       0.49      0.48      0.48      2419
weighted avg       0.52      0.53      0.52      2419
 samples avg       0.32      0.31      0.30      2419



In [60]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   13.5s remaining:    4.5s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   16.8s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   17.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   17.2s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.01, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.520264835269807


In [61]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=0.01,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4621, Recall: 0.4944, F1-measure: 0.4777
              precision    recall  f1-score   support

   flashback       0.26      0.42      0.32       549
      murder       0.56      0.60      0.58      1043
    violence       0.56      0.41      0.47       827

   micro avg       0.46      0.49      0.48      2419
   macro avg       0.46      0.48      0.46      2419
weighted avg       0.49      0.49      0.49      2419
 samples avg       0.28      0.29      0.27      2419



# CountVectorizing with chars and three,four grams:

In [62]:
vectorize=CountVectorizer(ngram_range=(3,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 93254)   (11027, 3)
(2730, 93254)   (2730, 3)


In [63]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   25.0s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   30.0s remaining:    6.6s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   31.3s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   33.0s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=2, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.5453978788963134


In [64]:
clf = OneVsRestClassifier(MultinomialNB(alpha=2),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5137, Recall: 0.5750, F1-measure: 0.5426
              precision    recall  f1-score   support

   flashback       0.28      0.31      0.30       549
      murder       0.62      0.65      0.64      1043
    violence       0.54      0.66      0.59       827

   micro avg       0.51      0.58      0.54      2419
   macro avg       0.48      0.54      0.51      2419
weighted avg       0.52      0.58      0.54      2419
 samples avg       0.30      0.33      0.30      2419



In [65]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   30.8s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   31.4s remaining:   10.5s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   38.3s remaining:    5.5s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   38.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   38.4s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1, average=False, class_weight=None, early_stopping=False,
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False),
          n_jobs=None))]
0.5168122291715993


In [66]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6302, Recall: 0.4452, F1-measure: 0.5218
              precision    recall  f1-score   support

   flashback       0.42      0.03      0.05       549
      murder       0.67      0.57      0.62      1043
    violence       0.59      0.56      0.57       827

   micro avg       0.63      0.45      0.52      2419
   macro avg       0.56      0.39      0.42      2419
weighted avg       0.59      0.45      0.48      2419
 samples avg       0.28      0.26      0.26      2419



In [67]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   22.4s remaining:    7.5s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   27.5s remaining:    3.9s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   27.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   27.7s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1, average=False, class_weight=None, early_stopping=False,
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False),
          n_jobs=None))]
0.5235888192652535


In [68]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6157, Recall: 0.4357, F1-measure: 0.5103
              precision    recall  f1-score   support

   flashback       0.36      0.05      0.09       549
      murder       0.67      0.55      0.61      1043
    violence       0.58      0.54      0.56       827

   micro avg       0.62      0.44      0.51      2419
   macro avg       0.54      0.38      0.42      2419
weighted avg       0.57      0.44      0.47      2419
 samples avg       0.28      0.26      0.25      2419



# CountVectorizing with chars and bi,tri,four grams:

In [69]:
vectorize=CountVectorizer(ngram_range=(2,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 93963)   (11027, 3)
(2730, 93963)   (2730, 3)


In [70]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   20.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   31.3s remaining:    6.9s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   33.9s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   35.5s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.5452721797775765


In [71]:
clf = OneVsRestClassifier(MultinomialNB(alpha=1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5034, Recall: 0.5862, F1-measure: 0.5416
              precision    recall  f1-score   support

   flashback       0.28      0.35      0.31       549
      murder       0.62      0.65      0.63      1043
    violence       0.53      0.66      0.59       827

   micro avg       0.50      0.59      0.54      2419
   macro avg       0.48      0.55      0.51      2419
weighted avg       0.51      0.59      0.55      2419
 samples avg       0.31      0.34      0.30      2419



In [72]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   33.3s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   34.0s remaining:   11.3s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   41.4s remaining:    5.9s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   41.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   41.8s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1, average=False, class_weight=None, early_stopping=False,
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False),
          n_jobs=None))]
0.5017684277401824


In [73]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6368, Recall: 0.3646, F1-measure: 0.4637
              precision    recall  f1-score   support

   flashback       0.34      0.05      0.08       549
      murder       0.64      0.62      0.63      1043
    violence       0.70      0.25      0.37       827

   micro avg       0.64      0.36      0.46      2419
   macro avg       0.56      0.31      0.36      2419
weighted avg       0.59      0.36      0.42      2419
 samples avg       0.26      0.21      0.22      2419



In [74]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   24.3s remaining:    8.1s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   30.2s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   30.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   30.4s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=2.5, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.518232839284313


In [75]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=2.5,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6955, Recall: 0.3154, F1-measure: 0.4340
              precision    recall  f1-score   support

   flashback       0.44      0.01      0.01       549
      murder       0.69      0.53      0.60      1043
    violence       0.71      0.25      0.37       827

   micro avg       0.70      0.32      0.43      2419
   macro avg       0.62      0.26      0.33      2419
weighted avg       0.64      0.32      0.39      2419
 samples avg       0.23      0.19      0.20      2419



# CountVectorizing with chars and one,bi,tri,four grams:

In [76]:
vectorize=CountVectorizer(ngram_range=(1,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 93990)   (11027, 3)
(2730, 93990)   (2730, 3)


In [77]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   26.7s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   30.7s remaining:    6.7s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   33.5s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   35.1s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.544813995058867


In [78]:
clf = OneVsRestClassifier(MultinomialNB(alpha=1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5030, Recall: 0.5858, F1-measure: 0.5413
              precision    recall  f1-score   support

   flashback       0.28      0.35      0.31       549
      murder       0.62      0.65      0.63      1043
    violence       0.53      0.66      0.59       827

   micro avg       0.50      0.59      0.54      2419
   macro avg       0.48      0.55      0.51      2419
weighted avg       0.51      0.59      0.55      2419
 samples avg       0.30      0.34      0.30      2419



In [79]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   17.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   33.6s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   33.9s remaining:   11.3s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   41.7s remaining:    6.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   42.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   42.6s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1, average=False, class_weight=None, early_stopping=False,
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False),
          n_jobs=None))]
0.4870044554276253


In [80]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.3853, Recall: 0.7627, F1-measure: 0.5119
              precision    recall  f1-score   support

   flashback       0.24      0.60      0.35       549
      murder       0.40      0.99      0.57      1043
    violence       0.56      0.59      0.57       827

   micro avg       0.39      0.76      0.51      2419
   macro avg       0.40      0.72      0.50      2419
weighted avg       0.42      0.76      0.52      2419
 samples avg       0.35      0.45      0.37      2419



In [81]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   24.0s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   24.4s remaining:    8.1s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   30.1s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   30.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   30.5s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.5068243651831236


In [82]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=0.001,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5086, Recall: 0.5713, F1-measure: 0.5382
              precision    recall  f1-score   support

   flashback       0.29      0.08      0.13       549
      murder       0.49      0.91      0.64      1043
    violence       0.60      0.48      0.53       827

   micro avg       0.51      0.57      0.54      2419
   macro avg       0.46      0.49      0.43      2419
weighted avg       0.49      0.57      0.49      2419
 samples avg       0.36      0.34      0.33      2419



In [83]:
from prettytable import PrettyTable
table = PrettyTable()
print("CountVectorization with word analyzer")
table.field_names=['Model','ngrams','f1_score','hyperparameter']
table.add_row(['MultinomialNB','1,1','0.535','1'])
table.add_row(['SGDClassifier(log)','1,1','0.504','0.1'])
table.add_row(['SGDClassifier(hinge)','1,1','0.507','0.1'])
table.add_row(['MultinomialNB','2,2','0.547','2'])
table.add_row(['SGDClassifier(log)','2,2','0.453','1e-05'])
table.add_row(['SGDClassifier(hinge)','2,2','0.479','1e-05'])
table.add_row(['MultinomialNB','3,3','0.384','1'])
table.add_row(['SGDClassifier(log)','3,3','0.375','1e-05'])
table.add_row(['SGDClassifier(hinge)','3,3','0.384','1e-05'])
table.add_row(['MultinomialNB','1,2','0.560','1'])
table.add_row(['SGDClassifier(log)','1,2','0.511','0.1'])
table.add_row(['SGDClassifier(hinge)','1,2','0.505','0.1'])
table.add_row(['MultinomialNB','1,3','0.560','1'])
table.add_row(['SGDClassifier(log)','1,3','0.496','0.01'])
table.add_row(['SGDClassifier(hinge)','1,3','0.486','0.1'])
print(table)
print("")
table = PrettyTable()
print("CountVectorization with char analyzer")
table.field_names=['Model','ngrams','f1_score','hyperparameter']
table.add_row(['MultinomialNB','3,3','0.536','2.5'])
table.add_row(['SGDClassifier(log)','3,3','0.438','2.5'])
table.add_row(['SGDClassifier(hinge)','3,3','0.493','0.0001'])
table.add_row(['MultinomialNB','4,4','0.546','2'])
table.add_row(['SGDClassifier(log)','4,4','0.539','0.1'])
table.add_row(['SGDClassifier(hinge)','4,4','0.477','0.1'])
table.add_row(['MultinomialNB','3,4','0.542','2'])
table.add_row(['SGDClassifier(log)','3,4','0.521','1'])
table.add_row(['SGDClassifier(hinge)','3,4','0.510','1'])
table.add_row(['MultinomialNB','2,4','0.541','1'])
table.add_row(['SGDClassifier(log)','2,4','0.463','1'])
table.add_row(['SGDClassifier(hinge)','2,4','0.434','2.5'])
table.add_row(['MultinomialNB','1,4','0.541','1'])
table.add_row(['SGDClassifier(log)','1,4','0.511','1'])
table.add_row(['SGDClassifier(hinge)','1,4','0.538','0.001'])
print(table)

CountVectorization with word analyzer
+----------------------+--------+----------+----------------+
|        Model         | ngrams | f1_score | hyperparameter |
+----------------------+--------+----------+----------------+
|    MultinomialNB     |  1,1   |  0.535   |       1        |
|  SGDClassifier(log)  |  1,1   |  0.504   |      0.1       |
| SGDClassifier(hinge) |  1,1   |  0.507   |      0.1       |
|    MultinomialNB     |  2,2   |  0.547   |       2        |
|  SGDClassifier(log)  |  2,2   |  0.453   |     1e-05      |
| SGDClassifier(hinge) |  2,2   |  0.479   |     1e-05      |
|    MultinomialNB     |  3,3   |  0.384   |       1        |
|  SGDClassifier(log)  |  3,3   |  0.375   |     1e-05      |
| SGDClassifier(hinge) |  3,3   |  0.384   |     1e-05      |
|    MultinomialNB     |  1,2   |  0.560   |       1        |
|  SGDClassifier(log)  |  1,2   |  0.511   |      0.1       |
| SGDClassifier(hinge) |  1,2   |  0.505   |      0.1       |
|    MultinomialNB     |  1,3   

# TfidfVectorizing with words and uni grams:

In [84]:
vectorize=TfidfVectorizer()
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 78666)   (11027, 3)
(2730, 78666)   (2730, 3)


In [85]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  35 out of  45 | elapsed:    3.0s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  40 out of  45 | elapsed:    3.3s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    3.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    3.5s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.4364152630710981


In [86]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))


Precision: 0.5526, Recall: 0.3344, F1-measure: 0.4167
              precision    recall  f1-score   support

   flashback       0.23      0.05      0.08       549
      murder       0.58      0.47      0.52      1043
    violence       0.58      0.36      0.44       827

   micro avg       0.55      0.33      0.42      2419
   macro avg       0.46      0.29      0.35      2419
weighted avg       0.50      0.33      0.39      2419
 samples avg       0.23      0.20      0.20      2419



In [87]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
grid_search_cv.best_score_

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    3.3s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    3.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    3.9s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]


0.4892576007667873

In [88]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5749, Recall: 0.4713, F1-measure: 0.5179
              precision    recall  f1-score   support

   flashback       0.33      0.15      0.21       549
      murder       0.62      0.60      0.61      1043
    violence       0.59      0.52      0.55       827

   micro avg       0.57      0.47      0.52      2419
   macro avg       0.52      0.42      0.46      2419
weighted avg       0.55      0.47      0.50      2419
 samples avg       0.29      0.27      0.27      2419



In [89]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    3.0s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    3.3s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    3.5s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.489679677184047


In [90]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5173, Recall: 0.4812, F1-measure: 0.4986
              precision    recall  f1-score   support

   flashback       0.29      0.25      0.27       549
      murder       0.59      0.59      0.59      1043
    violence       0.56      0.50      0.53       827

   micro avg       0.52      0.48      0.50      2419
   macro avg       0.48      0.45      0.46      2419
weighted avg       0.51      0.48      0.50      2419
 samples avg       0.29      0.28      0.27      2419



# TfidfVectorizing with words and bi grams

In [91]:
vectorize=TfidfVectorizer(ngram_range=(2,2),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 89722)   (11027, 3)
(2730, 89722)   (2730, 3)


In [92]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  13 out of  25 | elapsed:    0.6s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  16 out of  25 | elapsed:    0.6s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:    0.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  22 out of  25 | elapsed:    0.8s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.9s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.5055246557637904


In [93]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6451, Recall: 0.3952, F1-measure: 0.4901
              precision    recall  f1-score   support

   flashback       0.35      0.02      0.04       549
      murder       0.67      0.56      0.61      1043
    violence       0.63      0.43      0.51       827

   micro avg       0.65      0.40      0.49      2419
   macro avg       0.55      0.34      0.39      2419
weighted avg       0.58      0.40      0.45      2419
 samples avg       0.26      0.22      0.23      2419



In [94]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    1.9s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    2.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    2.2s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.46025958215263174


In [95]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6127, Recall: 0.3721, F1-measure: 0.4630
              precision    recall  f1-score   support

   flashback       0.40      0.08      0.13       549
      murder       0.65      0.49      0.56      1043
    violence       0.60      0.41      0.49       827

   micro avg       0.61      0.37      0.46      2419
   macro avg       0.55      0.33      0.39      2419
weighted avg       0.58      0.37      0.44      2419
 samples avg       0.25      0.21      0.21      2419



In [96]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    1.6s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    1.8s remaining:    0.1s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4660066000980762


[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    1.8s finished


In [97]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5687, Recall: 0.4088, F1-measure: 0.4757
              precision    recall  f1-score   support

   flashback       0.36      0.15      0.21       549
      murder       0.62      0.54      0.58      1043
    violence       0.57      0.41      0.48       827

   micro avg       0.57      0.41      0.48      2419
   macro avg       0.52      0.37      0.42      2419
weighted avg       0.54      0.41      0.46      2419
 samples avg       0.26      0.23      0.23      2419



# TfidfVectorizing with words and tri grams

In [98]:
vectorize=TfidfVectorizer(ngram_range=(3,3),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 5037)   (11027, 3)
(2730, 5037)   (2730, 3)


In [99]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1337s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  25 | elapsed:    0.2s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  10 out of  25 | elapsed:    0.3s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  13 out of  25 | elapsed:    0.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  16 out of  25 | elapsed:    0.3s remaining:    0.2s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.34812966571443943


[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:    0.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.5s finished


In [100]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5199, Recall: 0.2323, F1-measure: 0.3211
              precision    recall  f1-score   support

   flashback       0.25      0.03      0.05       549
      murder       0.54      0.36      0.43      1043
    violence       0.53      0.20      0.29       827

   micro avg       0.52      0.23      0.32      2419
   macro avg       0.44      0.20      0.26      2419
weighted avg       0.47      0.23      0.30      2419
 samples avg       0.16      0.13      0.14      2419



In [101]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0706s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  16 out of  55 | elapsed:    0.2s remaining:    0.5s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.3792980337124993


[Parallel(n_jobs=-1)]: Done  28 out of  55 | elapsed:    0.4s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  40 out of  55 | elapsed:    0.4s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    0.5s finished


In [103]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.3870, Recall: 0.3675, F1-measure: 0.3770
              precision    recall  f1-score   support

   flashback       0.23      0.21      0.22       549
      murder       0.45      0.43      0.44      1043
    violence       0.41      0.39      0.40       827

   micro avg       0.39      0.37      0.38      2419
   macro avg       0.36      0.35      0.35      2419
weighted avg       0.39      0.37      0.38      2419
 samples avg       0.22      0.21      0.20      2419



In [104]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0475s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  55 | elapsed:    0.1s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  16 out of  55 | elapsed:    0.2s remaining:    0.4s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.3786773413863728


[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    0.4s finished


In [105]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.3886, Recall: 0.3382, F1-measure: 0.3616
              precision    recall  f1-score   support

   flashback       0.26      0.25      0.25       549
      murder       0.46      0.38      0.42      1043
    violence       0.39      0.34      0.37       827

   micro avg       0.39      0.34      0.36      2419
   macro avg       0.37      0.32      0.35      2419
weighted avg       0.39      0.34      0.36      2419
 samples avg       0.21      0.19      0.19      2419



# TfidfVectorizing with words and uni,bi grams

In [106]:
vectorize=TfidfVectorizer(ngram_range=(1,2),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 109114)   (11027, 3)
(2730, 109114)   (2730, 3)


In [107]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  13 out of  25 | elapsed:    1.2s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  16 out of  25 | elapsed:    1.4s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:    1.8s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  22 out of  25 | elapsed:    2.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    2.4s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.47334183970721905


In [108]:
from sklearn.metrics import classification_report
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6404, Recall: 0.3799, F1-measure: 0.4769
              precision    recall  f1-score   support

   flashback       0.33      0.03      0.05       549
      murder       0.66      0.54      0.59      1043
    violence       0.64      0.41      0.50       827

   micro avg       0.64      0.38      0.48      2419
   macro avg       0.54      0.33      0.38      2419
weighted avg       0.58      0.38      0.44      2419
 samples avg       0.26      0.22      0.23      2419



In [109]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    5.4s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    5.9s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    6.1s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_i...e, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'clf__estimator__alpha': [3, 3.5, 2, 2.5, 1, 0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=10)

In [110]:
print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.49637189894788586


In [111]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5952, Recall: 0.4266, F1-measure: 0.4970
              precision    recall  f1-score   support

   flashback       0.34      0.13      0.19       549
      murder       0.64      0.56      0.60      1043
    violence       0.62      0.45      0.52       827

   micro avg       0.60      0.43      0.50      2419
   macro avg       0.53      0.38      0.44      2419
weighted avg       0.56      0.43      0.48      2419
 samples avg       0.28      0.25      0.24      2419



In [112]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    4.7s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    5.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    5.6s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.49755181673777343


In [113]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5187, Recall: 0.4580, F1-measure: 0.4865
              precision    recall  f1-score   support

   flashback       0.30      0.25      0.27       549
      murder       0.60      0.54      0.57      1043
    violence       0.55      0.49      0.52       827

   micro avg       0.52      0.46      0.49      2419
   macro avg       0.48      0.43      0.45      2419
weighted avg       0.52      0.46      0.49      2419
 samples avg       0.28      0.27      0.26      2419



# TfidfVectorizing with words and uni,bi,tri grams 

In [114]:
vectorize=TfidfVectorizer(ngram_range=(1,3),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 114151)   (11027, 3)
(2730, 114151)   (2730, 3)


In [115]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [2,1.5,1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  24 out of  35 | elapsed:    2.2s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  28 out of  35 | elapsed:    2.9s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  32 out of  35 | elapsed:    3.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    3.5s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.4744861067474211


In [116]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6422, Recall: 0.3836, F1-measure: 0.4803
              precision    recall  f1-score   support

   flashback       0.33      0.03      0.05       549
      murder       0.66      0.55      0.60      1043
    violence       0.64      0.41      0.50       827

   micro avg       0.64      0.38      0.48      2419
   macro avg       0.54      0.33      0.38      2419
weighted avg       0.58      0.38      0.44      2419
 samples avg       0.26      0.22      0.23      2419



In [117]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    5.6s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    6.2s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    6.5s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.5006802661353171


In [118]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5995, Recall: 0.4295, F1-measure: 0.5005
              precision    recall  f1-score   support

   flashback       0.34      0.12      0.17       549
      murder       0.65      0.55      0.60      1043
    violence       0.60      0.49      0.54       827

   micro avg       0.60      0.43      0.50      2419
   macro avg       0.53      0.38      0.44      2419
weighted avg       0.56      0.43      0.48      2419
 samples avg       0.28      0.25      0.25      2419



In [119]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    5.0s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    5.6s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    5.7s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4985304088558513


In [120]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5304, Recall: 0.4795, F1-measure: 0.5037
              precision    recall  f1-score   support

   flashback       0.33      0.28      0.30       549
      murder       0.60      0.59      0.60      1043
    violence       0.56      0.48      0.51       827

   micro avg       0.53      0.48      0.50      2419
   macro avg       0.50      0.45      0.47      2419
weighted avg       0.53      0.48      0.50      2419
 samples avg       0.29      0.28      0.27      2419



# TfidfVectorizing with words and uni,bi,tri,four grams

In [121]:
vectorize=TfidfVectorizer(ngram_range=(1,4),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 114710)   (11027, 3)
(2730, 114710)   (2730, 3)


In [122]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [2,1.5,1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)
print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  24 out of  35 | elapsed:    3.5s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  28 out of  35 | elapsed:    3.8s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  32 out of  35 | elapsed:    4.4s remaining:    0.4s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.47423785493452547


[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    4.8s finished


In [123]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6431, Recall: 0.3836, F1-measure: 0.4806
              precision    recall  f1-score   support

   flashback       0.33      0.03      0.05       549
      murder       0.66      0.56      0.60      1043
    violence       0.64      0.41      0.50       827

   micro avg       0.64      0.38      0.48      2419
   macro avg       0.54      0.33      0.38      2419
weighted avg       0.58      0.38      0.44      2419
 samples avg       0.26      0.22      0.23      2419



In [124]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    5.0s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    5.6s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    5.7s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.5052271685141727


In [125]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5526, Recall: 0.4585, F1-measure: 0.5011
              precision    recall  f1-score   support

   flashback       0.32      0.21      0.25       549
      murder       0.63      0.58      0.60      1043
    violence       0.57      0.47      0.52       827

   micro avg       0.55      0.46      0.50      2419
   macro avg       0.51      0.42      0.46      2419
weighted avg       0.54      0.46      0.49      2419
 samples avg       0.29      0.26      0.26      2419



In [126]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    4.7s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    5.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    5.4s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.5068177925053681


In [127]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5095, Recall: 0.4981, F1-measure: 0.5038
              precision    recall  f1-score   support

   flashback       0.32      0.25      0.28       549
      murder       0.58      0.58      0.58      1043
    violence       0.52      0.56      0.54       827

   micro avg       0.51      0.50      0.50      2419
   macro avg       0.47      0.46      0.47      2419
weighted avg       0.50      0.50      0.50      2419
 samples avg       0.29      0.28      0.27      2419



# TfidfVectorizing with char and tri grams

In [128]:
vectorize=TfidfVectorizer(ngram_range=(3,3),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 10726)   (11027, 3)
(2730, 10726)   (2730, 3)


In [129]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)



Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:    5.2s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:    6.2s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    6.4s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.0001, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.26614803669886655


In [130]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.0001),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6591, Recall: 0.1807, F1-measure: 0.2836
              precision    recall  f1-score   support

   flashback       0.26      0.01      0.02       549
      murder       0.68      0.30      0.41      1043
    violence       0.65      0.15      0.24       827

   micro avg       0.66      0.18      0.28      2419
   macro avg       0.53      0.15      0.22      2419
weighted avg       0.58      0.18      0.26      2419
 samples avg       0.14      0.11      0.12      2419



In [131]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    5.4s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    6.6s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    6.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    6.9s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.49143601615576715


In [132]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5148, Recall: 0.5684, F1-measure: 0.5403
              precision    recall  f1-score   support

   flashback       0.33      0.21      0.26       549
      murder       0.55      0.73      0.63      1043
    violence       0.53      0.60      0.56       827

   micro avg       0.51      0.57      0.54      2419
   macro avg       0.47      0.51      0.48      2419
weighted avg       0.49      0.57      0.52      2419
 samples avg       0.32      0.33      0.31      2419



In [133]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    4.7s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    5.8s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    6.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    6.0s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.49663593910050385


In [134]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5110, Recall: 0.4986, F1-measure: 0.5047
              precision    recall  f1-score   support

   flashback       0.32      0.20      0.25       549
      murder       0.60      0.55      0.57      1043
    violence       0.49      0.63      0.55       827

   micro avg       0.51      0.50      0.50      2419
   macro avg       0.47      0.46      0.46      2419
weighted avg       0.50      0.50      0.49      2419
 samples avg       0.30      0.29      0.27      2419



# TfidfVectorizing with char and four grams

In [135]:
vectorize=TfidfVectorizer(ngram_range=(4,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)


(11027, 82528)   (11027, 3)
(2730, 82528)   (2730, 3)


In [136]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   10.1s remaining:    3.4s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   11.8s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   12.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   12.6s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.4420720269807661


In [137]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))


Precision: 0.6094, Recall: 0.3489, F1-measure: 0.4437
              precision    recall  f1-score   support

   flashback       0.23      0.03      0.05       549
      murder       0.63      0.49      0.55      1043
    violence       0.62      0.39      0.48       827

   micro avg       0.61      0.35      0.44      2419
   macro avg       0.49      0.30      0.36      2419
weighted avg       0.54      0.35      0.41      2419
 samples avg       0.24      0.21      0.21      2419



In [138]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   15.3s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   17.3s remaining:    3.8s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   18.9s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   20.3s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.5097971885171857


In [139]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5810, Recall: 0.4419, F1-measure: 0.5020
              precision    recall  f1-score   support

   flashback       0.35      0.18      0.24       549
      murder       0.63      0.55      0.59      1043
    violence       0.61      0.47      0.53       827

   micro avg       0.58      0.44      0.50      2419
   macro avg       0.53      0.40      0.45      2419
weighted avg       0.56      0.44      0.49      2419
 samples avg       0.28      0.26      0.25      2419



In [140]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   15.9s remaining:    3.5s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   16.7s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   17.9s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-07, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4927641554524032


In [141]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-07,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5224, Recall: 0.4299, F1-measure: 0.4717
              precision    recall  f1-score   support

   flashback       0.28      0.18      0.22       549
      murder       0.59      0.55      0.57      1043
    violence       0.55      0.44      0.49       827

   micro avg       0.52      0.43      0.47      2419
   macro avg       0.47      0.39      0.43      2419
weighted avg       0.51      0.43      0.46      2419
 samples avg       0.27      0.25      0.25      2419



# TfidfVectorizing with char and tri,four grams

In [142]:
vectorize=TfidfVectorizer(ngram_range=(3,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 93254)   (11027, 3)
(2730, 93254)   (2730, 3)


In [143]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   22.7s remaining:    5.0s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   23.8s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   24.9s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.4493192086479685


In [144]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6047, Recall: 0.3605, F1-measure: 0.4517
              precision    recall  f1-score   support

   flashback       0.24      0.03      0.05       549
      murder       0.64      0.51      0.56      1043
    violence       0.60      0.40      0.48       827

   micro avg       0.60      0.36      0.45      2419
   macro avg       0.49      0.31      0.36      2419
weighted avg       0.53      0.36      0.42      2419
 samples avg       0.25      0.21      0.22      2419



In [145]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   20.1s remaining:    6.7s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   25.1s remaining:    3.6s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   25.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   25.7s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4900702374952076


In [146]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5800, Recall: 0.4539, F1-measure: 0.5093
              precision    recall  f1-score   support

   flashback       0.36      0.18      0.24       549
      murder       0.61      0.60      0.60      1043
    violence       0.63      0.45      0.53       827

   micro avg       0.58      0.45      0.51      2419
   macro avg       0.53      0.41      0.46      2419
weighted avg       0.56      0.45      0.49      2419
 samples avg       0.28      0.26      0.26      2419



In [147]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7,10**-8,10**-9],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   16.7s remaining:    5.6s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   20.6s remaining:    2.9s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   20.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   20.8s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-09, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.5000366630175914


In [149]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-09,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4841, Recall: 0.5176, F1-measure: 0.5003
              precision    recall  f1-score   support

   flashback       0.29      0.25      0.27       549
      murder       0.58      0.54      0.56      1043
    violence       0.48      0.67      0.56       827

   micro avg       0.48      0.52      0.50      2419
   macro avg       0.45      0.49      0.46      2419
weighted avg       0.48      0.52      0.49      2419
 samples avg       0.30      0.30      0.28      2419



# TfidfVectorizing with char and bi,tri,four grams

In [150]:
vectorize=TfidfVectorizer(ngram_range=(2,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 93963)   (11027, 3)
(2730, 93963)   (2730, 3)


In [151]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   24.0s remaining:    5.3s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   25.6s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   27.2s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.4334337353218364


In [152]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6189, Recall: 0.3357, F1-measure: 0.4353
              precision    recall  f1-score   support

   flashback       0.27      0.02      0.04       549
      murder       0.64      0.48      0.55      1043
    violence       0.63      0.36      0.46       827

   micro avg       0.62      0.34      0.44      2419
   macro avg       0.51      0.29      0.35      2419
weighted avg       0.55      0.34      0.40      2419
 samples avg       0.24      0.20      0.20      2419



In [153]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   27.6s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   29.1s remaining:    6.4s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   34.4s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   36.7s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-07, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4839952971562758


In [154]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-07,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5130, Recall: 0.4572, F1-measure: 0.4835
              precision    recall  f1-score   support

   flashback       0.31      0.12      0.18       549
      murder       0.59      0.46      0.52      1043
    violence       0.50      0.67      0.57       827

   micro avg       0.51      0.46      0.48      2419
   macro avg       0.47      0.42      0.42      2419
weighted avg       0.50      0.46      0.46      2419
 samples avg       0.29      0.27      0.26      2419



In [155]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   29.3s remaining:    6.4s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   30.6s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   32.9s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-07, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.5062054032918752


In [156]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-07,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5570, Recall: 0.4543, F1-measure: 0.5005
              precision    recall  f1-score   support

   flashback       0.40      0.04      0.07       549
      murder       0.60      0.57      0.58      1043
    violence       0.52      0.58      0.55       827

   micro avg       0.56      0.45      0.50      2419
   macro avg       0.51      0.40      0.40      2419
weighted avg       0.53      0.45      0.46      2419
 samples avg       0.29      0.26      0.26      2419



# TfidfVectorizing with char and uni,bi,tri,four grams

In [157]:
vectorize=TfidfVectorizer(ngram_range=(1,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 93990)   (11027, 3)
(2730, 93990)   (2730, 3)


In [158]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)
print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   17.2s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   24.9s remaining:    5.5s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   27.1s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   28.4s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.3225946011216488


In [159]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.001),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6552, Recall: 0.2356, F1-measure: 0.3466
              precision    recall  f1-score   support

   flashback       0.21      0.01      0.02       549
      murder       0.67      0.36      0.47      1043
    violence       0.67      0.22      0.34       827

   micro avg       0.66      0.24      0.35      2419
   macro avg       0.52      0.20      0.28      2419
weighted avg       0.57      0.24      0.32      2419
 samples avg       0.18      0.14      0.15      2419



In [160]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   28.0s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   33.2s remaining:    7.3s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   34.1s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   37.0s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.5000461094011961


In [161]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5420, Recall: 0.5014, F1-measure: 0.5209
              precision    recall  f1-score   support

   flashback       0.32      0.32      0.32       549
      murder       0.59      0.74      0.66      1043
    violence       0.69      0.32      0.44       827

   micro avg       0.54      0.50      0.52      2419
   macro avg       0.53      0.46      0.47      2419
weighted avg       0.56      0.50      0.51      2419
 samples avg       0.31      0.30      0.29      2419



In [162]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   27.6s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   32.1s remaining:    7.0s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   32.3s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   34.8s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-07, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.5095352331004768


In [163]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-07,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6068, Recall: 0.3324, F1-measure: 0.4295
              precision    recall  f1-score   support

   flashback       0.37      0.07      0.11       549
      murder       0.61      0.56      0.58      1043
    violence       0.66      0.23      0.34       827

   micro avg       0.61      0.33      0.43      2419
   macro avg       0.55      0.28      0.35      2419
weighted avg       0.58      0.33      0.39      2419
 samples avg       0.24      0.20      0.21      2419



In [164]:
from prettytable import PrettyTable
table = PrettyTable()
print("TfidfVectorization with word analyzer")
table.field_names=['Model','ngrams','f1_score','hyperparameter']
table.add_row(['MultinomialNB','1,1','0.416','0.01'])
table.add_row(['SGDClassifier(log)','1,1','0.517','1e-05'])
table.add_row(['SGDClassifier(hinge)','1,1','0.498','1e-05'])
table.add_row(['MultinomialNB','2,2','0.490','0.1'])
table.add_row(['SGDClassifier(log)','2,2','0.463','1e-06'])
table.add_row(['SGDClassifier(hinge)','2,2','0.475','1e-06'])
table.add_row(['MultinomialNB','3,3','0.321','0.01'])
table.add_row(['SGDClassifier(log)','3,3','0.377','1e-06'])
table.add_row(['SGDClassifier(hinge)','3,3','0.361','1e-06'])
table.add_row(['MultinomialNB','1,2','0.476','0.01'])
table.add_row(['SGDClassifier(log)','1,2','0.497','1e-05'])
table.add_row(['SGDClassifier(hinge)','1,2','0.486','1e-06'])
table.add_row(['MultinomialNB','1,3','0.480','0.01'])
table.add_row(['SGDClassifier(log)','1,3','0.500','1e-05'])
table.add_row(['SGDClassifier(hinge)','1,3','0.503','1e-05'])
table.add_row(['MultinomialNB','1,4','0.480','0.01'])
table.add_row(['SGDClassifier(log)','1,4','0.501','1e-06'])
table.add_row(['SGDClassifier(hinge)','1,4','0.503','1e-05'])
print(table)
print('')
table = PrettyTable()
print("TfidfVectorization with char analyzer")
table.field_names=['Model','ngrams','f1_score','hyperparameter']
table.add_row(['MultinomialNB','3,3','0.283','0.0001'])
table.add_row(['SGDClassifier(log)','3,3','0.540','1e-05'])
table.add_row(['SGDClassifier(hinge)','3,3','0.504','1e-05'])
table.add_row(['MultinomialNB','4,4','0.443','0.01'])
table.add_row(['SGDClassifier(log)','4,4','0.502','1e-05'])
table.add_row(['SGDClassifier(hinge)','4,4','0.471','1e-07'])
table.add_row(['MultinomialNB','3,4','0.451','0.01'])
table.add_row(['SGDClassifier(log)','3,4','0.509','1e-05'])
table.add_row(['SGDClassifier(hinge)','3,4','0.500','1e-09'])
table.add_row(['MultinomialNB','2,4','0.435','0.01'])
table.add_row(['SGDClassifier(log)','2,4','0.435','0.01'])
table.add_row(['SGDClassifier(hinge)','2,4','0.500','1e-07'])
table.add_row(['MultinomialNB','1,4','0.283','0.001'])
table.add_row(['SGDClassifier(log)','1,4','0.346','1e-05'])
table.add_row(['SGDClassifier(hinge)','1,4','0.429','1e-07'])
print(table)

TfidfVectorization with word analyzer
+----------------------+--------+----------+----------------+
|        Model         | ngrams | f1_score | hyperparameter |
+----------------------+--------+----------+----------------+
|    MultinomialNB     |  1,1   |  0.416   |      0.01      |
|  SGDClassifier(log)  |  1,1   |  0.517   |     1e-05      |
| SGDClassifier(hinge) |  1,1   |  0.498   |     1e-05      |
|    MultinomialNB     |  2,2   |  0.490   |      0.1       |
|  SGDClassifier(log)  |  2,2   |  0.463   |     1e-06      |
| SGDClassifier(hinge) |  2,2   |  0.475   |     1e-06      |
|    MultinomialNB     |  3,3   |  0.321   |      0.01      |
|  SGDClassifier(log)  |  3,3   |  0.377   |     1e-06      |
| SGDClassifier(hinge) |  3,3   |  0.361   |     1e-06      |
|    MultinomialNB     |  1,2   |  0.476   |      0.01      |
|  SGDClassifier(log)  |  1,2   |  0.497   |     1e-05      |
| SGDClassifier(hinge) |  1,2   |  0.486   |     1e-06      |
|    MultinomialNB     |  1,3   